# Parcours Ingénieur IA
## Projet 6 : Améliorez le produit IA de votre start-up
## Etudiant : Alwis VARRAT

##**Extraction de données avec l'API Yelp**

In [5]:
! pip install load_dotenv 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install yelp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yelp: filename=yelp-1.0.2-py3-none-any.whl size=11803 sha256=41b15013ec20941050751e62b9d57673a08099586eb3ac316764ed0e5bdba20c
  Stored in directory: /root/.cache/pip/wheels/b6/1e/49/279cb99bdd89a75d1e14543916e015238f50a47a70bd49076b
Successfully built yelp


In [7]:
from yelp.client import Client
import requests
import requests
import pandas as pd
import uuid
import json
from dotenv import load_dotenv

In [8]:
load_dotenv()

False

**Définir lees paramètres de l'API (header, query...)**

In [9]:
#Initialisation du client
MY_API_KEY = "GPRDoyrsk_uUv5j_LfnMN4ZqFKzpIIocBuWIS6GXvRvxC6vESMTth4dx0yTsrY7x8igpc2yTZzlwzjt77ufwuaJvE09eqQj5xqNVx-6h4jQesbXi6DEs7mEYDsk-Y3Yx" #  Replace this with your real API key

client = Client(MY_API_KEY)

In [10]:
# Endpoint de la requête API
api_url = "https://api.yelp.com/v3/graphql"

headers = {
    "Authorization": F"Bearer {MY_API_KEY}",
    "Content-Type": "application/graphql",
    "charsete": "UTF-8"
}

In [11]:
# définir les paramètres de la requête
query = F'{{\n\
        search(\
            categories: "Restaurants", \
            location: "paris", \
            offset: 0, \
            limit:  50\
        ) {{\n\
            business {{\n\
                id\n\
                name\n\
                rating\n\
                is_closed\n\
                reviews {{\n\
                    id\n\
                    user {{\n\
                        id\n\
                    }}\n\
                    text\n\
                    rating\n\
                }}\n\
            }}\n\
        }}\n\
    }}'

In [12]:
#récuperation de la reponse
response = requests.post(api_url, headers=headers, data=query)
response_json = response.json()
print(response.status_code)
# Pretty Print JSON
json_formatted_str = json.dumps(response_json, indent=4)
print(json_formatted_str)

200
{
    "data": {
        "search": {
            "business": [
                {
                    "id": "qq-0DAkiLPJivqCJff6nFw",
                    "name": "Mus\u00e9e du Louvre",
                    "rating": 4.5,
                    "is_closed": false,
                    "reviews": [
                        {
                            "id": "0Wx3zdZBn_4vFh7F0GdrLw",
                            "user": {
                                "id": "MafVtb7iRfcujHBKcPFORQ"
                            },
                            "text": "Merveilleuse! J'ai ador\u00e9 les belles \u0153uvres d'art trouv\u00e9es dans tout le mus\u00e9e. Je l'ai tellement aim\u00e9, c'est la troisi\u00e8me fois que j'y vais cette semaine.",
                            "rating": 5
                        },
                        {
                            "id": "E0gYDSrfw19uLhhe5Hcx6A",
                            "user": {
                                "id": "svrexz-HiSUt4LsFR6pj3Q"
         

In [13]:
#définir les champs à récuperer
businesses_dict = {
    'business_id': [],
    'business_name': [],
    'business_stars': [],
    'business_is_closed': [],
    'reviews': []
}

**Récuperation de données avec les paramètres définis**

In [14]:
for offset in range(0, 200, 50):
    
    response = requests.post(api_url, headers=headers, data=query)
    response_code = response.status_code

    if response_code == 200:
        response_json = response.json()
        for business in response_json.get("data", {}).get("search", {}).get("business", []):
            businesses_dict['business_id'].append(business.get("id"))
            businesses_dict['business_name'].append(business.get("name"))
            businesses_dict['business_stars'].append(business.get("rating"))
            businesses_dict['business_is_closed'].append(business.get("is_closed"))
            businesses_dict['reviews'].append(business.get("reviews", []))    
    else : 
        print("erreur : " + str(response_code) + " : " + response.text)

In [15]:
#convertir le dico vers pandas Dataframe
businesses_df = pd.DataFrame.from_dict(businesses_dict)

In [16]:
businesses_df.shape

(200, 5)

In [17]:
businesses_df.head()

,business_id,business_name,business_stars,business_is_closed,reviews
0,qq-0DAkiLPJivqCJff6nFw,Musée du Louvre,4.5,False,"[{'id': '0Wx3zdZBn_4vFh7F0GdrLw', 'user': {'id..."
1,In3z-xigGXnYXlBS-mhOPg,La Tour Eiffel,4.5,False,"[{'id': 'BMID8zgqjhMsV14dCCNa5A', 'user': {'id..."
2,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,4.5,False,"[{'id': 'lyhANPEYoAlF14drJmU2lQ', 'user': {'id..."
3,FFz-WusZrBYZexKqhqzCkg,L'As du Fallafel,4.5,False,"[{'id': '8ye4l1-aR7EzpiMh6JdtOQ', 'user': {'id..."
4,XlMMUoGIOhIE-aWRXgTd1A,Musée d’Orsay,4.5,False,"[{'id': 'JKy_GoXP-ERzCXpxQcIQug', 'user': {'id..."


In [19]:
#export de la donnée en csv 
businesses_df.to_csv('data/api_businesses_df.csv')